# Boundary Value Problem

---

In the [Classical Physics Models](http://nbviewer.jupyter.org/github/JuliaDiffEq/DiffEqTutorials.jl/blob/master/PhysicalModels/ClassicalPhysics.ipynb) tutorial there is a simple pendulum example. We solved it as an initial value problem a.k.a. IVP, i.e. I know the initial position and velocity, and tell me what will happen. In this notebook, we will solve the pendulum problem when we know the conditions different time. This kind of problem is known as boundary value problem a.k.a. BVP.

In [16]:
using Plots, BoundaryValueDiffEq, OrdinaryDiffEq; gr()

Plots.GRBackend()

Everything in this code block will be the same as the IVP one.

In [9]:
#Constants
const g = 9.81
L = 1.0

#Initial guess
u₀ = [0,π/2]
tspan = (0.0,pi/2)

#Define the problem
function simplependulum(t,u,du)
    θ  = u[1]
    dθ = u[2]
    du[1] = dθ
    du[2] = -(g/L)*sin(θ)
end

simplependulum (generic function with 1 method)

Here is where the `Boundary` comes in. We need to write a function that calculate the residual in-place from the problem solution, such that the residual is $\vec{0}$ when the boundary condition is satisfied.

In [11]:
function bc1(residual, u)
    residual[1] = u[end÷2][1] + pi/2 # the solution at the middle of the time span should be -pi/2
    residual[2] = u[end][1] - pi/2 # the solution at the end of the time span should be pi/2
end

bvp1 = BVProblem(simplependulum, bc1, u₀, tspan)
sol1 = solve(bvp1, GeneralMIRK4(), dt=0.05)
plot(sol1)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 -2.5 
 
 
 0.0 
 
 
 2.5 
 
 
 t 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.3701,210.673 56.6437,219.223 73.9173,227.216 91.1909,234.517 108.465,241.029 125.738,246.686 143.012,251.455 160.285,255.32 177.559,258.279 194.833,260.338 
 212.106,261.502 229.38,261.776 246.654,261.162 263.927,259.657 281.201,257.256 298.474,253.951 315.748,249.74 333.022,244.629 350.295,238.64 367.569,231.817 
 384.843,224.238 402.116,216.015 419.39,207.295 436.663,198.263 453.937,189.123 471.211,180.091 488.484,171.372 505.758,163.148 523.031,155.569 540.305,148.747 
 557.579,142.758 574.852,137.647 592.126,133.436 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e26f46; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.3701,372.441 56.6437,362.725 73.9173,349.875 91.1909,334.653 108.465,317.805 125.738,299.975 143.012,281.659 160.285,263.196 177.559,244.781 194.833,226.496 
 212.106,208.334 229.38,190.238 246.654,172.127 263.927,153.922 281.201,135.584 298.474,117.136 315.748,98.6985 333.022,80.5196 350.295,62.9978 367.569,46.6883 
 384.843,32.2809 402.116,20.5403 419.39,12.2068 436.663,7.87402 453.937,7.87402 471.211,12.2068 488.484,20.5403 505.758,32.2809 523.031,46.6883 540.305,62.9978 
 557.579,80.5196 574.852,98.6985 592.126,117.136 
 "/>
 
 
 
 
 u1(t) 
 
 
 
 u2(t)

We have boundary conditions at the beginning and the ending of the time span in common cases. We can use the `TwoPointBVProblem` problem type for such cases.

In [12]:
function bc2(residual, ua, ub) # ua is the beginning of the time span, and ub is the ending
    residual[1] = ua[1] + pi/2     # the solution at the beginning of the time span should be -pi/2
    residual[2] = ub[1] - pi/2 # the solution at the end of the time span should be pi/2
end
bvp2 = TwoPointBVProblem(simplependulum, bc2, u₀, tspan)
sol2 = solve(bvp2, MIRK4(), dt=0.05) # we need to use the MIRK4 solver for TwoPointBVProblem
plot(sol2)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 t 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.3701,364.436 56.6437,368.431 73.9173,371.093 91.1909,372.429 108.465,372.441 125.738,371.13 143.012,368.493 160.285,364.523 177.559,359.218 194.833,352.583 
 212.106,344.645 229.38,335.454 246.654,325.104 263.927,313.732 281.201,301.531 298.474,288.743 315.748,275.65 333.022,262.558 350.295,249.77 367.569,237.569 
 384.843,226.197 402.116,215.847 419.39,206.656 436.663,198.718 453.937,192.083 471.211,186.778 488.484,182.808 505.758,180.171 523.031,178.86 540.305,178.872 
 557.579,180.208 574.852,182.87 592.126,186.865 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e26f46; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.3701,370.638 56.6437,343.445 73.9173,316.355 91.1909,289.372 108.465,262.434 125.738,235.453 143.012,208.366 160.285,181.173 177.559,153.991 194.833,127.09 
 212.106,100.938 229.38,76.2212 246.654,53.8296 263.927,34.8028 281.201,20.215 298.474,11.0185 315.748,7.87402 333.022,11.0185 350.295,20.215 367.569,34.8028 
 384.843,53.8296 402.116,76.2212 419.39,100.938 436.663,127.09 453.937,153.991 471.211,181.173 488.484,208.366 505.758,235.453 523.031,262.434 540.305,289.372 
 557.579,316.355 574.852,343.445 592.126,370.638 
 "/>
 
 
 
 
 u1(t) 
 
 
 
 u2(t)

We have used the mono-implicit Runge–Kutta (MIRK) method to solve the BVP, but we can always use reduce a BVP to an IVP and a root-finding problem, which is the `Shooting` method. If you can have a good initial guess, Shooting method works very well.

In [17]:
u₀_2 = [-1.6, -1.7]
sol3 = solve(bvp1, Shooting(Vern7()))
plot(sol3)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 -2.5 
 
 
 0.0 
 
 
 2.5 
 
 
 t 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.3701,224.081 45.5808,227.111 51.7916,230.06 58.0023,232.922 64.213,235.691 70.4238,238.359 76.6345,240.924 82.8453,243.38 89.056,245.723 95.2667,247.95 
 101.477,250.058 107.688,252.044 113.899,253.908 120.11,255.646 126.32,257.258 132.531,258.743 138.742,260.1 144.953,261.328 151.163,262.427 157.374,263.397 
 163.585,264.238 169.796,264.949 176.006,265.53 182.217,265.982 188.428,266.305 194.639,266.498 200.849,266.561 207.06,266.495 213.271,266.299 219.482,265.974 
 225.692,265.52 231.903,264.935 238.114,264.222 244.325,263.378 250.535,262.406 256.746,261.304 262.957,260.073 269.167,258.713 275.378,257.226 281.589,255.611 
 287.8,253.87 294.01,252.004 300.221,250.015 306.432,247.905 312.643,245.675 318.853,243.33 325.064,240.872 331.275,238.305 337.486,235.634 343.696,232.864 
 349.907,230 356.118,227.049 362.329,224.017 368.539,220.913 374.75,217.743 380.961,214.517 387.172,211.243 393.382,207.932 399.593,204.592 405.804,201.234 
 412.015,197.868 418.225,194.505 424.436,191.154 430.647,187.825 436.857,184.529 443.068,181.276 449.279,178.074 455.49,174.932 461.7,171.859 467.911,168.862 
 474.122,165.949 480.333,163.126 486.543,160.399 492.754,157.774 498.965,155.255 505.176,152.846 511.386,150.552 517.597,148.375 523.808,146.318 530.019,144.384 
 536.229,142.573 542.44,140.888 548.651,139.329 554.862,137.899 561.072,136.596 567.283,135.423 573.494,134.378 579.705,133.463 585.915,132.677 592.126,132.021 
 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e26f46; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.3701,372.441 45.5808,368.079 51.7916,363.311 58.0023,358.173 64.213,352.698 70.4238,346.921 76.6345,340.875 82.8453,334.594 89.056,328.108 95.2667,321.446 
 101.477,314.636 107.688,307.701 113.899,300.664 120.11,293.545 126.32,286.361 132.531,279.127 138.742,271.855 144.953,264.556 151.163,257.238 157.374,249.908 
 163.585,242.572 169.796,235.232 176.006,227.892 182.217,220.552 188.428,213.214 194.639,205.876 200.849,198.54 207.06,191.203 213.271,183.865 219.482,176.527 
 225.692,169.187 231.903,161.847 238.114,154.507 244.325,147.171 250.535,139.842 256.746,132.524 262.957,125.226 269.167,117.954 275.378,110.721 281.589,103.538 
 287.8,96.4201 294.01,89.3852 300.221,82.4529 306.432,75.6452 312.643,68.9869 318.853,62.5049 325.064,56.2283 331.275,50.1882 337.486,44.417 343.696,38.9486 
 349.907,33.8174 356.118,29.058 362.329,24.7044 368.539,20.7895 374.75,17.3443 380.961,14.3974 387.172,11.974 393.382,10.0954 399.593,8.77873 405.804,8.03595 
 412.015,7.87402 418.225,8.29434 424.436,9.29305 430.647,10.861 436.857,12.9839 443.068,15.6427 449.279,18.8141 455.49,22.4715 461.7,26.5849 467.911,31.1226 
 474.122,36.0511 480.333,41.3362 486.543,46.9434 492.754,52.8386 498.965,58.9879 505.176,65.3594 511.386,71.9231 517.597,78.6509 523.808,85.5168 530.019,92.4973 
 536.229,99.5712 542.44,106.72 548.651,113.927 554.862,121.179 561.072,128.463 567.283,135.771 573.494,143.095 579.705,150.428 585.915,157.766 592.126,165.106 
 
 "/>
 
 
 
 
 u1(t) 
 
 
 
 u2(t)